In [3]:
from ipywidgets import widgets
from ipywidgets import interactive
from IPython.display import display, HTML
from pivottablejs import pivot_ui

import csv
import numpy as np
import pandas as pd
import os, sys
import subprocess
import base64 #for createdownloadlink

data_folder = '../new-data/'
dci_data_folder = data_folder+'DCI/all-combined/'
dfs_afcars_data_file = data_folder+'DFS/DFS_12-16-AFCARS-CLEAN.csv'
dfs_county_data_file = data_folder+'DFS/DFS_12-16-Placements-ByCounty-CLEAN.csv'
dfs_plc_data_file = data_folder+'DFS/DFS_12-16-Placements-ByPLC-CLEAN.csv'
school_discipline_data_file = data_folder+'school-discipline/SchoolDiscipline_2007-17_Combined-CSV.csv'

#found:https://stackoverflow.com/questions/31893930/download-csv-from-an-ipython-notebook
def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)
#end found

#Functions to get data frames
def getOverview():#DONE
    gather_overview_frames = []
    current_path = dci_data_folder+'overview/'
    year_files = next(os.walk(current_path))[2]
    for f in year_files:#process each file for this year
        file_path = current_path+'/'+f
        year = f.split('-')[0]
        try:
            df = pd.read_csv(file_path,sep=',',header='infer',index_col=0)
            df.assign(Year=year)
            gather_overview_frames.append(df)
        except Exception as e:
            print('cant load file: '+file_path)
            print('['+str(e)+']')
            print('*****************************************************************************************')
    return pd.concat(gather_overview_frames)

def getDFS():
    afcars = pd.read_csv(dfs_afcars_data_file,sep=',',header='infer',index_col=0)
    county = pd.read_csv(dfs_county_data_file,sep=',',header='infer',index_col=0)
    plc = pd.read_csv(dfs_plc_data_file,sep=',',header='infer',index_col=0)
    return afcars, county, plc

def getSchool():
    return pd.read_csv(school_discipline_data_file,sep=',',header='infer',index_col=0)


In [16]:
overview=getOverview()
school = getSchool()
school['County']=school['DISTRICT_NAME'].str.split('#').str[0]
#new = school[school['DISTRICT_NAME']]
display(school)
# display(HTML(overview.to_html()))

,DISTRICT_NAME,Beginning Year,End Year,Expulsion - Services not provided,Expulsion - Services provided,In school suspension,Out of school suspension,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,County
DISTRICT_ID,,,,,,,,,,,,,,,,,,,,,
101000,Albany #1,2007,2008,NaN,NaN,35.0,154.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Albany
201000,Big Horn #1,2007,2008,NaN,NaN,22.0,23.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Big Horn
202000,Big Horn #2,2007,2008,NaN,NaN,2.0,45.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Big Horn
203000,Big Horn #3,2007,2008,NaN,NaN,74.0,39.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Big Horn
204000,Big Horn #4,2007,2008,NaN,NaN,7.0,16.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Big Horn
301000,Campbell #1,2007,2008,1.0,NaN,259.0,55.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Campbell
401000,Carbon #1,2007,2008,NaN,NaN,429.0,282.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carbon
402000,Carbon #2,2007,2008,NaN,NaN,29.0,20.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carbon
501000,Converse #1,2007,2008,NaN,NaN,41.0,45.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Converse


In [36]:
#def getIndexOffenses():
import matplotlib.pyplot as plt
county_drop = widgets.Dropdown(
    options=school.County.unique().tolist(),
    description='County:',
    disabled=False,
)

def county_changed(change):
    print('county changed to:'+str(change['new']))
    print(school.loc[change['new'] in school['DISTRICT_NAME']])
    countyDF = school[school['DISTRICT_NAME'].str.match(change['new'])]
    noServices = countyDF['Expulsion - Services not provided'].sum()
    services = countyDF['Expulsion - Services provided'].sum()
    #countyDF.plot(x='Beginning Year',y='Out of school suspension')
    
county_drop.observe(county_changed, names='value')

display(county_drop)

Dropdown(description='County:', options=('Albany ', 'Big Horn ', 'Campbell ', 'Carbon ', 'Converse ', 'Crook '…

county changed to:Campbell 


KeyError: 'the label [False] is not in the [index]'